In [1]:
import pandas as pd
data = pd.read_csv('/content/df_labeled_filtered.csv')
data.head(10)
data = data.replace({'label': {'n': 1, 'p':2, '0':0}})
print (data.shape)
data = data.drop_duplicates(subset=["sentence"])
print (data.shape)
min_rows = min(len(data[data['label'] == 2]),len(data[data['label'] == 1]),len(data[data['label'] == 0]))
trimmed_data = data.groupby('label').head(min_rows)
print (trimmed_data.shape)
print(trimmed_data['label'].value_counts())
data=trimmed_data

# plt.hist(trimmed_data)


(12871, 3)
(12871, 3)
(6198, 3)
label
1    2066
0    2066
2    2066
Name: count, dtype: int64


<ipython-input-1-5926d40faedf>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({'label': {'n': 1, 'p':2, '0':0}})


In [ ]:
#for the twitter sentiment anaalysis testing
import pandas as pd
data = pd.read_csv('/content/test.csv', encoding='latin1')
data = data.replace({'sentiment': {'negative': 1, 'positive':2, 'neutral':0}})
# data = data.drop_duplicates(subset=["sentence"], keep=False)
data ['label'] = data['sentiment'].copy()
data ['sentence' ] = data ['text'].copy()
data.head(10)

<ipython-input-1-8facb87ec5ea>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({'sentiment': {'negative': 1, 'positive':2, 'neutral':0}})


,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),label,sentence
0,1,Last session of the day http://twitpic.com/67ezh,0,morning,0-20,Afghanistan,38928346,652860.0,60,0,Last session of the day http://twitpic.com/67ezh
1,2,Shanghai is also really exciting (precisely -...,2,noon,21-30,Albania,2877797,27400.0,105,2,Shanghai is also really exciting (precisely -...
2,3,"Recession hit Veronique Branquinho, she has to...",1,night,31-45,Algeria,43851044,2381740.0,18,1,"Recession hit Veronique Branquinho, she has to..."
3,4,happy bday!,2,morning,46-60,Andorra,77265,470.0,164,2,happy bday!
4,5,http://twitpic.com/4w75p - I like it!!,2,noon,60-70,Angola,32866272,1246700.0,26,2,http://twitpic.com/4w75p - I like it!!
5,6,that`s great!! weee!! visitors!,2,night,70-100,Antigua and Barbuda,97929,440.0,223,2,that`s great!! weee!! visitors!
6,7,I THINK EVERYONE HATES ME ON HERE lol,1,morning,0-20,Argentina,45195774,2736690.0,17,1,I THINK EVERYONE HATES ME ON HERE lol
7,8,"soooooo wish i could, but im in school and my...",1,noon,21-30,Armenia,2963243,28470.0,104,1,"soooooo wish i could, but im in school and my..."
8,9,and within a short time of the last clue all ...,0,night,31-45,Australia,25499884,7682300.0,3,0,and within a short time of the last clue all ...
9,10,What did you get? My day is alright.. haven`...,0,morning,46-60,Austria,9006398,82400.0,109,0,What did you get? My day is alright.. haven`...


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


# BERT: No training or finetuning


In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split


data['label'] = data['label'].astype(int)


# Assuming your data has 'sentence' and 'label' columns
texts = data['sentence'].tolist()
labels = data['label'].tolist()

# Split the data into train (80%) and test (20%) sets
_, X_test, _, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Load pretrained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # Update num_labels for your classes
model.to(device)

# Preprocess data for BERT (batch processing for large data)
batch_size = 8  # Adjust as needed based on GPU memory

# Prepare inputs in batches
predictions = []
model.eval()
with torch.no_grad():
    for i in range(0, len(X_test), batch_size):
        batch_texts = X_test[i:i + batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=128)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        batch_predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        predictions.extend(batch_predictions)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy: 0.35161290322580646
F1 Score: 0.18539647031266845


In [5]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.35161290322580646
F1 Score: 0.18539647031266845
Precision: 0.23069918614175425
Recall: 0.35161290322580646


#BERT: With fine tuned data from our dataset


In [6]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score

data['label'] = data['label'].astype(int)

# Assuming your data has 'sentence' and 'label' columns
texts = data['sentence'].tolist()
labels = data['label'].tolist()

# Split the data into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load pretrained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
train_encodings = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt', max_length=128)
test_encodings = tokenizer(X_test, padding=True, truncation=True, return_tensors='pt', max_length=128)

# Convert labels to tensors
train_labels = torch.tensor(y_train)
test_labels = torch.tensor(y_test)

# Create DataLoader for training and testing
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)

# Load pretrained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.to(device)

# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * 3  # Assuming 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Fine-tuning the model
model.train()
for epoch in range(3):  # Train for 3 epochs
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

# Evaluation on the test set
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(batch_predictions)
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Accuracy: 0.8201612903225807
F1 Score: 0.8197413130305429


In [8]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.8201612903225807
F1 Score: 0.8197413130305429
Precision: 0.821675262284293
Recall: 0.8201612903225807


In [ ]:
#to visualize mismatches

import matplotlib.pyplot as plt

mismatched_indices = [i for i in range(len(true_labels)) if true_labels[i] != predictions[i]]
mismatched_true = [true_labels[i] for i in mismatched_indices]
mismatched_predictions = [predictions[i] for i in mismatched_indices]

plt.figure(figsize=(10, 6))
plt.scatter(mismatched_indices, mismatched_true, label='True Labels', color='blue', marker='o')
plt.scatter(mismatched_indices, mismatched_predictions, label='Predictions', color='red', marker='x')

plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Mismatched Predictions vs True Labels')
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
#if we want to go through some mismatched sentences

# Identify mismatched indices
mismatched_indices = [i for i in range(len(true_labels)) if true_labels[i] != predictions[i]]

# Display sentences, correct labels, and incorrect predictions for mismatches
for i in mismatched_indices:
    print(f"Sentence: {texts[i]}")
    print(f"True Label: {true_labels[i]}")
    print(f"Predicted Label: {predictions[i]}")
    print("-" * 40)


#ROBERTA: No training


In [13]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

_, X_test, _, y_test = train_test_split(data['sentence'], data['label'], test_size=0.2, random_state=42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the pretrained RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
model.to(device)

# Preprocess data for RoBERTa (batch processing for large data)
batch_size = 8  # Adjust as needed based on GPU memory

# Prepare inputs in batches and make predictions
predictions = []
model.eval()
with torch.no_grad():
    for i in range(0, len(X_test), batch_size):
        batch_texts = X_test[i:i + batch_size].tolist()
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=128)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        batch_predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        predictions.extend(batch_predictions)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy: 0.3233870967741935
F1 Score: 0.16433168033593945


In [14]:
output_dir = '/content/saved_models/RoBERTanoFT'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model and tokenizer have been saved to {output_dir}")


Model and tokenizer have been saved to /content/saved_models/RoBERTanoFT


In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.3233870967741935
F1 Score: 0.16433168033593945
Precision: 0.13531045693258548
Recall: 0.3233870967741935


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#ROBERTA: Fine tuned with our dataset

In [16]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['label'], test_size=0.2, random_state=42)

# Load the pretrained RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the training and test data
train_encodings = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)
test_encodings = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Convert labels to tensors
train_labels = torch.tensor(y_train.values)
test_labels = torch.tensor(y_test.values)

# Create DataLoader for training and testing
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=8)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=8)

# Load the pretrained RoBERTa model for sequence classification
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
model.to(device)

# Set up the optimizer and learning rate scheduler for fine-tuning
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * 3  # Assuming 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Fine-tuning loop
model.train()
epochs = 3  # Number of epochs

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Average training loss: {avg_train_loss}')

# Evaluation on the test set
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(batch_predictions)
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
Average training loss: 0.6328776809236696
Epoch 2/3
Average training loss: 0.3494099697158221
Epoch 3/3
Average training loss: 0.17774809651647605
Accuracy: 0.8290322580645161
F1 Score: 0.8287254396283521


In [17]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.8290322580645161
F1 Score: 0.8287254396283521
Precision: 0.8301303496874889
Recall: 0.8290322580645161


In [18]:
output_dir = '/content/drive/MyDrive/RoBERTawFT'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model and tokenizer have been saved to {output_dir}")


Model and tokenizer have been saved to /content/drive/MyDrive/RoBERTawFT


#XLNet without **finetuning**

In [20]:
# Install necessary packages
!pip install -q transformers pandas scikit-learn torch

import torch
import pandas as pd
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Check if GPU is available and enable it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Split the data into train (80%) and test (20%) sets
_, X_test, _, y_test = train_test_split(data['sentence'], data['label'], test_size=0.2, random_state=42)

# Load the pretrained XLNet tokenizer and model
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=3)
model.to(device)

# Preprocess data for XLNet (batch processing for large data)
batch_size = 8  # Adjust as needed based on GPU memory
predictions = []

# Put the model in evaluation mode
model.eval()

# Prepare the test data and make predictions
with torch.no_grad():
    for i in range(0, len(X_test), batch_size):
        batch_texts = X_test[i:i + batch_size].tolist()
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=128)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        # Get predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        batch_predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        predictions.extend(batch_predictions)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')


Using device: cuda


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy: 0.34516129032258064
F1 Score: 0.24771558342839645


In [21]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.34516129032258064
F1 Score: 0.24771558342839645
Precision: 0.3170299639726454
Recall: 0.34516129032258064


#XLNet with finetuning

In [ ]:
# Install necessary packages
!pip install -q transformers pandas scikit-learn torch

import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Check if GPU is available and enable it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



# Split the data into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['label'], test_size=0.2, random_state=42)

# Load the pretrained XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Tokenize the training and test data
train_encodings = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)
test_encodings = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Convert labels to tensors
train_labels = torch.tensor(y_train.values)
test_labels = torch.tensor(y_test.values)

# Create DataLoader for training and testing
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=8)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=8)

# Load the pretrained XLNet model for sequence classification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=3)
model.to(device)

# Set up the optimizer and learning rate scheduler for fine-tuning
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * 3  # Assuming 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
model.train()
epochs = 3  # Number of epochs

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Average training loss: {avg_train_loss}')

# Evaluation on the test set
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(batch_predictions)
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')


Using device: cuda


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
Average training loss: 0.693888140914421
Epoch 2/3
Average training loss: 0.41457690827007737
Epoch 3/3


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
